<a href="https://colab.research.google.com/github/ciherraiz/guita/blob/master/SPY_Weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!pip install investpy

In [85]:
import datetime
from google.colab import files
import investpy
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [86]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [87]:
FILENAME='SPY.csv'
NAME = 'SPDR S&P 500'
COUNTRY = 'united states'
FROM_DATE = datetime.datetime(1990, 1, 1)
TO_DATE = datetime.datetime.now()

In [88]:
# Strategy parameters
DTE = 10 # (5 = 1 week)
WINDOW_SMA = 7
FROM_FILTER_DATE = datetime.datetime(2019, 6, 30)
TO_FILTER_DATE = datetime.datetime.now()

In [89]:
investpy.etfs.get_etf_information(etf=NAME, country=COUNTRY)

,ETF Name,Prev. Close,Todays Range,ROI (TTM),Open,52 wk Range,Dividends (TTM),Volume,Market Cap,Dividend Yield,Average Vol. (3m),Total Assets,Beta,1-Year Change,Shares Outstanding,Asset Class
0,SPDR S&P 500,307.35,299.42 - 306.39,- 0.46%,306.16,218.26 - 339.08,4.03,127961017.0,2.666200e+11,1.91%,116658477.0,1.676500e+11,1.0,2.93%,892432116.0,Equity


In [90]:
data = investpy.etfs.get_etf_historical_data(etf='SPDR S&P 500',
                                             country='united states',
                                             from_date=FROM_DATE.strftime('%d/%m/%Y'),
                                             to_date=TO_DATE.strftime('%d/%m/%Y'))
data.to_csv(FILENAME)
#files.download(FILENAME)

In [91]:
data['Week day'] = data.index.dayofweek + 1
data['Day name'] = data.index.day_name()

In [92]:
data['Close SMA']=data['Close'].rolling(window=WINDOW_SMA).mean()

In [93]:
data[['Close DTE', 'Close SMA DTE']] = data[['Close', 'Close SMA']].shift(periods=DTE-1)

In [94]:
data['Return'] = data['Close'] - data['Close DTE']

In [95]:
data['Trigger'] = data['Close DTE'] < data['Close SMA DTE']

In [96]:
data.dropna(inplace=True)

In [97]:
expirations = data['Week day'].isin([1,3,5])

In [98]:
data_exp = data[expirations]

In [99]:
data_exp

,Open,High,Low,Close,Currency,Exchange,Week day,Day name,Close SMA,Close DTE,Close SMA DTE,Return,Trigger
Date,,,,,,,,,,,,,
1993-02-24,43.72,44.25,43.72,44.25,USD,NYSE,3,Wednesday,43.648571,44.72,44.781429,-0.47,True
1993-02-26,44.44,44.44,44.19,44.41,USD,NYSE,5,Friday,43.911429,44.59,44.835714,-0.18,True
1993-03-01,44.56,44.56,44.22,44.28,USD,NYSE,1,Monday,44.035714,43.47,44.617143,0.81,True
1993-03-03,45.00,45.16,44.94,45.12,USD,NYSE,3,Wednesday,44.432857,43.41,44.175714,1.71,True
1993-03-05,44.94,45.12,44.72,44.75,USD,NYSE,5,Friday,44.674286,43.72,43.875714,1.03,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-17,314.04,314.37,311.04,311.66,USD,NYSE,3,Wednesday,310.897143,319.34,309.114286,-7.68,False
2020-06-19,314.17,314.38,306.53,308.64,USD,NYSE,5,Friday,308.130000,323.20,314.150000,-14.56,False
2020-06-22,307.95,311.04,306.81,310.62,USD,NYSE,1,Monday,309.560000,320.79,316.327143,-10.17,False


In [100]:
n_expirations = len(data_exp)
n_triggered = len(data_exp[data_exp["Trigger"]==True])
n_triggered_win = len(data_exp[(data_exp["Trigger"]==True) & (data_exp['Return']>0)])

In [101]:
print(f'Total expiration dates {n_expirations}')
print(f'Total triggered {n_triggered}')
print(f'Total triggered win {n_triggered_win} ({n_triggered_win/n_triggered:.2%})')

Total expiration dates 4089
Total triggered 1695
Total triggered win 1044 (61.59%)


In [102]:
data_plot = data[FROM_FILTER_DATE : TO_FILTER_DATE].copy()

In [103]:
#fig = px.line(data, x=data.index, y="Close", color='continent')
fig = go.Figure(data=go.Scatter(x=data_plot.index, y=data_plot['Close'], mode='lines', line_color='rgba(200, 200, 200, 1)'))
#fig.add_trace(go.Scatter(x=data_exp.index, y=data['Trigger'], mode='markers', marker_color='rgba(255, 182, 193, .9)'))
fig.show()

In [107]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_plot.index, y=data_plot['Close'], mode='markers', marker_color='rgba(255, 182, 193, 1)'))
fig.add_trace(go.Scatter(x=data_plot.index, y=data_plot[data_plot['Trigger']==True]['Close'], mode='markers', marker_color='rgba(0, 182, 0, 1)'))

In [109]:
data_plot['Close Win'] = data_plot[data_plot['Return']>0]['Close']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [111]:
data_plot['Close Win'] = data_plot.loc[data_plot['Return']>0, 'Close']

SyntaxError: ignored

In [116]:
data_plot['Close Win'] = data_plot[data_plot['Trigger'] ==True]['Close']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

